In [4]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

In [49]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="tesi",
    vectors_config=VectorParams(size=384, distance=Distance.DOT),
)

True

In [58]:
from fastembed import TextEmbedding
from fastembed.common.model_description import PoolingType, ModelSource

sentences = ["Hello how are you?"]

model = TextEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings = list(model.embed(sentences))



Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00,  8.73it/s]


In [1]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

def chunk_text(text, max_length=1000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = start + max_length
        chunks.append(text[start:end])
        start += max_length - overlap
    return chunks

In [ ]:
import json
import re
pdf_path = "Studi/Dead_Code.pdf"
text = extract_text_from_pdf(pdf_path)


'''chunks = chunk_text(text, max_length=1000, overlap=200)


for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:\n{chunk[:100]}...\n")'''

['Dead Code\nSimone Romano\nUniversity of Basilicata\nPotenza, Italy\nsimone.romano@unibas.it\nAbstract —Dead code is a bad smell. It is conjectured to be\nharmful and it appears to be also a common phenomenon in\nsoftware systems. Surprisingly, dead code has received little\nempirical attention from the software engineering research com-\nmunity. This post-doctoral track paper shows the main results of\na multi-study investigation into dead code with an overarching\ngoal to study when and why developers introduce dead code,\nhow they perceive and cope with it, and whether dead code\nis harmful. This investigation is composed of semi-structured\ninterviews with software professionals and four experiments at\nthe University of Basilicata and the College of William & Mary', 'The results suggest that it is worth studying dead code not only in\nmaintenance and evolution phases, where the results suggest that\nits presence is detrimental to developers, but also in design and\nimplementation

'chunks = chunk_text(text, max_length=1000, overlap=200)\n\n\nfor i, chunk in enumerate(chunks):\n    print(f"Chunk {i+1}:\n{chunk[:100]}...\n")'

In [3]:
from docling.chunking import HierarchicalChunker
from docling.document_converter import DocumentConverter
chunker = HierarchicalChunker()
converter = DocumentConverter()
doc = converter.convert("Studi/Dead_Code.pdf").document
chunks = chunker.chunk(doc)
for chunk in chunks:
    print(chunk)

c:\Users\ACER\Desktop\tesi\semantic search SLR\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


text='Simone Romano University of Basilicata Potenza, Italy simone.romano@unibas.it' meta=DocMeta(schema_name='docling_core.transforms.chunker.DocMeta', version='1.0.0', doc_items=[TextItem(self_ref='#/texts/2', parent=RefItem(cref='#/body'), children=[], content_layer=<ContentLayer.BODY: 'body'>, label=<DocItemLabel.TEXT: 'text'>, prov=[ProvenanceItem(page_no=1, bbox=BoundingBox(l=246.487, t=640.0940146484376, r=342.133, b=597.0540146484375, coord_origin=<CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>), charspan=(0, 77))], orig='Simone Romano University of Basilicata Potenza, Italy simone.romano@unibas.it', text='Simone Romano University of Basilicata Potenza, Italy simone.romano@unibas.it', formatting=None, hyperlink=None)], headings=['Dead Code'], captions=None, origin=DocumentOrigin(mimetype='application/pdf', binary_hash=15418977965400417667, filename='Dead_Code.pdf', uri=None))
text='Abstract -Dead code is a bad smell. It is conjectured to be harmful and it appears to be also a common phe

In [45]:
pdf_embeddings = list(model.embed(chunks))
pdf_embeddings

[array([-7.46973124e-02, -1.36553086e-02, -9.14183261e-02, -3.29932964e-02,
         1.04822497e-02, -7.39636514e-02, -7.32132895e-03,  2.89358512e-02,
         9.44347042e-03,  5.80315679e-03, -1.23262230e-02,  4.91105596e-02,
         5.68274932e-02, -6.73444096e-02, -1.68260332e-02,  2.11940416e-03,
        -7.17146460e-02, -4.20647531e-02, -3.31332420e-02, -6.36253119e-03,
         3.06387302e-03,  9.95953380e-03, -3.27217929e-02,  3.24966942e-02,
         4.47770970e-02,  1.09410781e-02, -3.59826262e-02, -9.12260259e-02,
         1.56649535e-03,  4.44730973e-02, -5.26427417e-03,  1.45972257e-01,
        -7.70761978e-03,  1.24489389e-02,  4.00188864e-02,  8.62229674e-02,
         2.90656008e-02, -6.46831768e-02, -4.62622463e-02,  3.73863889e-02,
        -1.57355342e-01,  8.49778486e-02, -1.36124541e-02,  8.73576379e-02,
         1.67634172e-02, -6.62688380e-02, -6.81048200e-02, -2.86824268e-02,
        -6.23604707e-02, -9.17961522e-02, -9.50507280e-02, -3.67354000e-02,
         5.0

In [ ]:
from qdrant_client.models import PointStruct

points = []

for i,c in enumerate(pdf_embeddings):
    points.append(PointStruct(id = i, vector=c, payload = {"title": pdf_path}))

operation_info = client.upsert(
    collection_name="tesi",
    wait=True,
    points=points,
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [63]:
question = "What are the primary causes of dead code in software systems?"
query = list(model.embed(question))

search_result = client.query_points(
    collection_name="tesi",
    query=query[0],
    with_payload=True,
    with_vectors=True,
    limit=3
).points

vectors = []
for res in search_result:
    with open("output.txt", "a", encoding="utf-8") as f:
        f.write(chunks[res.id] + "\n\nvector break\n\n")

